In [6]:
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

True

In [7]:
import torch
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from openai import OpenAI

# 경로/모델 설정
DB_DIR = "./chroma_db"
COLLECTION_NAME = "qna_collection"
EMBED_MODEL = "BAAI/bge-m3"
OPENAI_MODEL = "gpt-4o-mini"
TOP_K = 5


embeddings = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL,
    encode_kwargs={"normalize_embeddings": True},
)

# 크로마 벡터스토어 로드
vs = Chroma(
    collection_name=COLLECTION_NAME,
    persist_directory=DB_DIR,
    embedding_function=embeddings,
)

# OpenAI 클라이언트
oclient = OpenAI()

print("Vector store loaded. Count:", vs._collection.count())

Vector store loaded. Count: 3536


In [8]:
retriever = vs.as_retriever(search_kwargs={"k": TOP_K})
test_docs = retriever.get_relevant_documents("BigQuery에서 새로운 데이터셋을 생성하는 방법은 무엇인가요?")
print("검색 결과 개수:", len(test_docs))
for i, d in enumerate(test_docs, 1):
    print(f"[{i}] source:", d.metadata.get("source") or d.metadata.get("sources"))
    print(d.page_content[:200].replace("\n"," "), "\n---")

검색 결과 개수: 5
[1] source: ["https://cloud.google.com/bigquery/docs/reference/rest"]
Q: BigQuery에서 새로운 데이터셋을 생성하는 방법은 무엇인가요? A: 새로운 데이터셋을 생성하려면 다음의 API 메서드를 사용합니다: `insert` 메서드. 요청은 다음과 같이 구성됩니다:  ``` POST /bigquery/v2/projects/{projectId}/datasets ``` 여기서 `{projectId}`는 데이터셋을 생성할 프로젝 
---
[2] source: ["https://cloud.google.com/bigquery/docs/reference/rest/v2/datasets/insert"]
Q: BigQuery에서 새로운 데이터셋을 생성하기 위한 HTTP 요청은 어떻게 구성하나요? A: 새로운 데이터셋을 생성하기 위한 HTTP 요청은 다음과 같이 구성됩니다:  ``` POST https://bigquery.googleapis.com/bigquery/v2/projects/{projectId}/datasets ``` 여기서 `{projectId} 
---
[3] source: ["https://cloud.google.com/bigquery/docs/reference/rest"]
Q: BigQuery에서 특정 데이터셋에 새로운 루틴을 생성하는 방법은 무엇인가요? A: 새로운 루틴을 생성하려면 다음의 REST API를 사용합니다: POST /bigquery/v2/projects/{projectId}/datasets/{datasetId}/routines 
---
[4] source: ["https://cloud.google.com/bigquery/docs/reference/rest/v2/datasets"]
Q: BigQuery 데이터셋에 대한 태그를 설정하는 방법은 무엇인가요? A: 데이터셋에 태그(resourceTags)를 추가하려면, 데이터셋을 생성할 때 해당 필드를 사용하여 태그를 지정해

In [17]:
def _show_sources(meta) -> str:

    s = meta.get("source")
    if not s:
        s = meta.get("sources")
        if isinstance(s, list):
            s = ", ".join([str(x) for x in s])
    return s or "(no source)"

def build_messages(user_query: str, docs):
    lines = []
    for i, d in enumerate(docs, start=1):
        src = _show_sources(d.metadata or {})
        body = d.page_content[:1200]
        lines.append(f"[{i}] source={src}\n{body}")
    context_block = "\n\n".join(lines) if lines else "(no context)"

    system_prompt = (
        "너는 Google API 문서 기반 한국어 어시스턴트다. "
        "아래 제공된 근거 스니펫 안에서 확인 가능한 내용만 답하고, 모르면 모른다고 말해라. "
        "고유 식별자/API/메서드/필드/에러명/옵션명은 원문 영문을 괄호로 병기하고, "
        "코드 블록은 수정/번역하지 말고 원문 그대로 인용해라."
    )

    user_prompt = f"""사용자 질문:
{user_query}

[근거 스니펫]
{context_block}

요구사항:
- 스니펫에서 직접 확인 가능한 사실만 사용.
- 스니펫에서 확인 불가하거나 불확실하면 '문서에서 확인 불가'라고 명시.
- 답변은 단순 요약이 아니라, 사용자가 이해하기 쉽게 **조금 더 설명**을 붙여라.
- 답변 끝에는 '참고' 섹션을 추가하여, 본문에서 실제로 인용한 근거 번호와 해당 SOURCE URL만 나열하라. (다른 메타데이터는 제외)
- 코드 블록이 있으면 그대로 인용하고, 필요시 짧은 해설을 추가하라.
- 최종 출력은 한국어.
"""
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

def rag_answer(query: str, top_k: int = TOP_K):
    # 1) 검색
    retriever = vs.as_retriever(search_kwargs={"k": top_k})
    docs = retriever.get_relevant_documents(query)

    # 2) LLM 호출
    messages = build_messages(query, docs)
    resp = oclient.chat.completions.create(
        model=OPENAI_MODEL,
        messages=messages,
        temperature=0.0,
    )
    answer = resp.choices[0].message.content
    return answer, docs

In [19]:
question = "BigQuery에서 새로운 데이터셋을 생성하는 방법은 무엇인가요?"
answer, evidences = rag_answer(question)

print("=== 질문 ===")
print(question)

print("\n=== 답변 ===")
print(answer.strip())

=== 질문 ===
BigQuery에서 새로운 데이터셋을 생성하는 방법은 무엇인가요?

=== 답변 ===
BigQuery에서 새로운 데이터셋을 생성하려면 `insert` 메서드를 사용해야 합니다. 이 메서드는 HTTP POST 요청을 통해 호출되며, 요청 URL은 다음과 같이 구성됩니다:

```
POST https://bigquery.googleapis.com/bigquery/v2/projects/{projectId}/datasets
```

여기서 `{projectId}`는 데이터셋을 생성할 프로젝트의 ID를 의미합니다. 이 요청을 통해 새로운 데이터셋이 해당 프로젝트 내에 생성됩니다. 데이터셋을 생성할 때는 추가적인 설정이나 속성을 포함할 수 있으며, 필요한 경우 태그를 지정할 수도 있습니다.

참고:
- [1] https://cloud.google.com/bigquery/docs/reference/rest
- [2] https://cloud.google.com/bigquery/docs/reference/rest/v2/datasets/insert
